# Project 08 — American Options Pricing (Longstaff–Schwartz) + Greeks (Interactive)

## Goal
Price an **American** option in two different ways and connect the theory to visuals:

- **CRR binomial tree** (benchmark, discrete-time no-arbitrage)
- **Longstaff–Schwartz Monte Carlo (LSM)** (regression-based optimal stopping)
- Compare prices, visualize **exercise region**, and inspect **Greeks** (finite differences)

We work under the standard risk-neutral GBM model:

\[
dS_t = r S_t\,dt + \sigma S_t\, dW_t
\]

In [1]:
from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go

import ipywidgets as widgets
from IPython.display import display

SEED = 123
rng = np.random.default_rng(SEED)

PROJECT_DIR = Path.cwd()
ASSETS_DIR = PROJECT_DIR / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", PROJECT_DIR)
print("ASSETS_DIR:", ASSETS_DIR.resolve())

CWD: c:\Users\Karim\Desktop\quant-finance-portfolio\projects
ASSETS_DIR: C:\Users\Karim\Desktop\quant-finance-portfolio\projects\assets


## 1) Helpers

### Simulate GBM paths
Using Euler on log-price:

\[
\log S_{t+\Delta t} = \log S_t + \left(r-\tfrac{1}{2}\sigma^2\right)\Delta t + \sigma\sqrt{\Delta t}\,Z
\quad (Z\sim\mathcal N(0,1))
\]

In [2]:
def simulate_gbm_paths(S0: float, r: float, sigma: float, T: float, n_steps: int, n_paths: int, rng: np.random.Generator):
    dt = T / n_steps
    t = np.linspace(0.0, T, n_steps + 1)
    Z = rng.standard_normal(size=(n_paths, n_steps))
    increments = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z

    logS = np.zeros((n_paths, n_steps + 1))
    logS[:, 0] = np.log(S0)
    logS[:, 1:] = logS[:, [0]] + np.cumsum(increments, axis=1)
    S = np.exp(logS)
    return t, S

def payoff(S: np.ndarray, K: float, option_type: str):
    if option_type.lower() == "call":
        return np.maximum(S - K, 0.0)
    elif option_type.lower() == "put":
        return np.maximum(K - S, 0.0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

### CRR binomial tree (American exercise)

We use the Cox–Ross–Rubinstein tree:

- \(u = e^{\sigma\sqrt{\Delta t}}\), \(d = 1/u\)
- Risk-neutral prob: \(p = \frac{e^{r\Delta t}-d}{u-d}\)

Backward induction with early exercise at each node:

\[
V = \max\big( \text{payoff},\ e^{-r\Delta t}(pV_u + (1-p)V_d)\big)
\]

In [3]:
def american_option_crr(S0: float, K: float, r: float, sigma: float, T: float, n_steps: int, option_type: str):
    dt = T / n_steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1.0 / u
    disc = np.exp(-r * dt)
    p = (np.exp(r * dt) - d) / (u - d)

    # terminal stock prices
    j = np.arange(n_steps + 1)
    S_T = S0 * (u ** j) * (d ** (n_steps - j))
    V = payoff(S_T, K, option_type)

    for i in range(n_steps - 1, -1, -1):
        j = np.arange(i + 1)
        S_i = S0 * (u ** j) * (d ** (i - j))
        cont = disc * (p * V[1:] + (1.0 - p) * V[:-1])
        ex = payoff(S_i, K, option_type)
        V = np.maximum(ex, cont)

    return float(V[0])

### Longstaff–Schwartz (LSM)

Key idea: at each time step \(t_i\), estimate the **continuation value**
\(\mathbb E[V_{i+1} \mid S_i]\) by regression on basis functions
(e.g., \(1, S, S^2\)). Then exercise if immediate payoff is better.

This produces an approximation of the optimal stopping rule.

In [4]:
def lsm_american_option(
    S0: float, K: float, r: float, sigma: float, T: float,
    n_steps: int, n_paths: int, option_type: str,
    degree: int = 2,
    rng: np.random.Generator | None = None,
):
    if rng is None:
        rng = np.random.default_rng(0)

    dt = T / n_steps
    disc = np.exp(-r * dt)

    t, S = simulate_gbm_paths(S0, r, sigma, T, n_steps, n_paths, rng)
    H = payoff(S, K, option_type)

    # cashflows: start from maturity payoffs
    cf = H[:, -1].copy()
    exercised = np.zeros_like(H, dtype=bool)
    exercised[:, -1] = H[:, -1] > 0

    # go backward in time (exclude t=0 and maturity)
    for i in range(n_steps - 1, 0, -1):
        in_money = H[:, i] > 0
        if not np.any(in_money):
            cf *= disc
            continue

        X = S[in_money, i]
        Y = cf[in_money] * disc  # discounted continuation from i+1 to i

        # polynomial regression basis
        A = np.vstack([X**k for k in range(degree + 1)]).T  # [1, X, X^2, ...]
        beta, *_ = np.linalg.lstsq(A, Y, rcond=None)
        cont = A @ beta

        ex_val = H[in_money, i]
        exercise_now = ex_val > cont

        # update cashflows: if exercise now, cashflow = payoff; else keep continuation
        idx = np.where(in_money)[0]
        ex_idx = idx[exercise_now]
        cf[ex_idx] = ex_val[exercise_now]
        exercised[ex_idx, i] = True

        # those not exercising keep future CF; discount everybody one step for next iteration
        cf *= disc

    price = float(np.mean(cf) * np.exp(-r * 0.0))  # cf already discounted back to t=0
    return price, t, S, H, exercised

## 2) Interactive pricing + visuals

What you will see:

- **Price comparison**: CRR vs LSM  
- **Exercise boundary plot**: where early exercise happens (by time)  
- **Payoff histogram**: asymmetry of option payoffs

In [5]:
opt_type_w = widgets.Dropdown(options=["put", "call"], value="put", description="type")
S0_w = widgets.FloatSlider(value=100, min=10, max=300, step=1, description="S0")
K_w  = widgets.FloatSlider(value=100, min=10, max=300, step=1, description="K")
r_w  = widgets.FloatSlider(value=0.03, min=0.0, max=0.10, step=0.001, description="r")
sig_w= widgets.FloatSlider(value=0.20, min=0.05, max=1.0, step=0.01, description="sigma")
T_w  = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="T")

steps_w = widgets.IntSlider(value=50, min=10, max=400, step=10, description="steps")
paths_w = widgets.IntSlider(value=30000, min=2000, max=200000, step=2000, description="paths")
deg_w   = widgets.IntSlider(value=2, min=1, max=5, step=1, description="poly deg")

btn = widgets.Button(description="Run pricing", button_style="success")
out = widgets.Output()

display(widgets.VBox([
    widgets.HBox([opt_type_w, S0_w, K_w]),
    widgets.HBox([r_w, sig_w, T_w]),
    widgets.HBox([steps_w, paths_w, deg_w]),
    btn,
    out
]))

In [6]:
def finite_diff_greeks(price_fn, S0, eps=0.01):
    # central difference delta and gamma (in S0)
    p_up = price_fn(S0 * (1 + eps))
    p_dn = price_fn(S0 * (1 - eps))
    p_0  = price_fn(S0)
    delta = (p_up - p_dn) / (2 * S0 * eps)
    gamma = (p_up - 2 * p_0 + p_dn) / ((S0 * eps) ** 2)
    return delta, gamma

def run(_=None):
    with out:
        out.clear_output()

        S0 = float(S0_w.value)
        K  = float(K_w.value)
        r  = float(r_w.value)
        sig= float(sig_w.value)
        T  = float(T_w.value)
        n_steps = int(steps_w.value)
        n_paths = int(paths_w.value)
        deg = int(deg_w.value)
        otype = str(opt_type_w.value)

        # CRR (fast, deterministic)
        crr = american_option_crr(S0, K, r, sig, T, n_steps, otype)

        # LSM (Monte Carlo)
        price_lsm, t, S, H, exercised = lsm_american_option(
            S0, K, r, sig, T, n_steps, n_paths, otype, degree=deg, rng=np.random.default_rng(SEED)
        )

        # Greeks (finite differences on LSM price)
        def lsm_price_at(S0_):
            p, *_ = lsm_american_option(S0_, K, r, sig, T, n_steps, n_paths, otype, degree=deg, rng=np.random.default_rng(SEED))
            return p

        delta, gamma = finite_diff_greeks(lsm_price_at, S0, eps=0.01)

        summary = pd.DataFrame({
            "Method": ["CRR (binomial)", "LSM (Monte Carlo)"],
            "Price": [crr, price_lsm],
        })
        display(summary.style.format({"Price":"{:.4f}"}))
        print(f"LSM finite-diff Delta ≈ {delta:.4f} | Gamma ≈ {gamma:.6f}")

        # --- Payoff distribution (discounted terminal payoff) ---
        disc_T = np.exp(-r * T)
        payoff_T = H[:, -1] * disc_T
        fig_pay = px.histogram(payoff_T, nbins=80, template="plotly_dark",
                               title="Discounted terminal payoff distribution")
        fig_pay.update_layout(xaxis_title="Discounted payoff", yaxis_title="Count")
        fig_pay.show()
        fig_pay.write_html(ASSETS_DIR / "payoff_hist.html")

        # --- Exercise boundary: for each time, show min/median/max S where exercise happened ---
        ex_points = []
        for i in range(1, n_steps):
            idx = exercised[:, i]
            if np.any(idx):
                ex_points.append((t[i], np.quantile(S[idx, i], 0.10), np.quantile(S[idx, i], 0.50), np.quantile(S[idx, i], 0.90)))
        if len(ex_points) > 0:
            ex_df = pd.DataFrame(ex_points, columns=["t", "S_q10", "S_q50", "S_q90"])
            fig_ex = go.Figure()
            fig_ex.add_trace(go.Scatter(x=ex_df["t"], y=ex_df["S_q50"], mode="lines+markers", name="Median exercise S"))
            fig_ex.add_trace(go.Scatter(x=ex_df["t"], y=ex_df["S_q10"], mode="lines", name="q10", line=dict(dash="dot")))
            fig_ex.add_trace(go.Scatter(x=ex_df["t"], y=ex_df["S_q90"], mode="lines", name="q90", line=dict(dash="dot")))
            fig_ex.update_layout(template="plotly_dark", title="LSM estimated exercise region (quantiles)",
                                 xaxis_title="Time", yaxis_title="Underlying price S")
            fig_ex.show()
            fig_ex.write_html(ASSETS_DIR / "exercise_boundary.html")
        else:
            print("No early exercise observed (often happens for calls with no dividends).")

        # --- Quick interpretation ---
        print("\nInterpretation tips:")
        print("- American puts often exercise early when deep in-the-money (interest on strike matters).")
        print("- American calls (no dividends) should rarely exercise early under Black–Scholes assumptions.")
        print("- If LSM and CRR disagree a lot: increase paths, increase steps, or adjust regression degree.")

btn.on_click(run)
run()

## What to say in interviews (short)

- Implemented **two** American pricing engines: **CRR** (no-arbitrage) and **LSM** (optimal stopping).
- Demonstrated why **early exercise** is economically meaningful (puts, rates, dividends).
- Built interactive visuals for **exercise regions** and payoff distributions.